# Comparación de Modelos de Sentimiento

Este notebook compara el rendimiento de un modelo de **Regresión Logística con TF-IDF** (entrenado localmente) contra un modelo de **Transformers pre-entrenado (RoBERTa)** de la librería Hugging Face, especializado en análisis de sentimiento en tweets.

In [1]:
import pandas as pd
import joblib
import re
from transformers import pipeline

pd.set_option('display.max_colwidth', 120)
pd.set_option('display.width', 200)

### Se define la función de limpieza de texto y una nueva función para encapsular la predicción con el modelo local.

In [2]:
import spacy
import os

# Cargar el modelo de spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print("Descargando modelo de spaCy...")
    os.system('python -m spacy download en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')


def limpieza(texto):
    """Realiza una limpieza completa del texto para el modelo local."""
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúüñ\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    if not texto:
        return ""
    doc = nlp(texto)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop])

def predecir_con_modelo_local(textos, vectorizer, modelo):
    """Encapsula todo el proceso de predicción del modelo local."""
    textos_limpios = [limpieza(t) for t in textos]
    textos_transformados = vectorizer.transform(textos_limpios)
    predicciones = modelo.predict(textos_transformados)
    
    mapa_local = {-1: "Negativo", 1: "Positivo"}
    return [mapa_local.get(p, "Desconocido") for p in predicciones]

### Carga, Predicción y Comparación

In [3]:
try:
    # --- Cargar Modelos ---
    print("Cargando el modelo local y el vectorizer...")
    vectorizer_cargado = joblib.load('model_robust/schimizzi_vectorizer_1.6kk.joblib')
    modelo_cargado = joblib.load('model_robust/schimizzi_modelo_1.6kk.joblib')
    print("Modelo local cargado!")

    print("\nCargando el modelo pre-entrenado de Hugging Face...")
    sentiment_pipeline_hf = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
    print("¡Modelo de Hugging Face cargado!")

    # --- Cargar Datos para Comparación ---
    ruta_archivo_nuevo = 'data/testdata.manual.2009.06.14.csv'
    df_comparacion = pd.read_csv(ruta_archivo_nuevo, header=None, encoding='latin-1')
    df_comparacion.columns = ['Sentiment_Real', 'id', 'date', 'query', 'user', 'Text']
    
    # Tomamos una muestra para no tardar demasiado
    textos_a_comparar = df_comparacion['Text'].sample(30).tolist()
    
    # --- Realizar Predicciones ---
    print("\nRealizando predicciones con ambos modelos...")
    
    predicciones_local = predecir_con_modelo_local(textos_a_comparar, vectorizer_cargado, modelo_cargado)
    resultados_hf = sentiment_pipeline_hf(textos_a_comparar)
    
    # --- Formatear Resultados y Mostrar Tabla ---
    mapa_hf = {'LABEL_0': 'Negativo', 'LABEL_1': 'Neutral', 'LABEL_2': 'Positivo'}
    predicciones_hf_texto = [mapa_hf.get(r['label'], "Desconocido") for r in resultados_hf]
    confianza_hf = [r['score'] for r in resultados_hf]

    df_resultados = pd.DataFrame({
        'Texto Original': textos_a_comparar,
        'Tu Modelo': predicciones_local,
        'Hugging Face': predicciones_hf_texto,
        'Confianza HF': confianza_hf
    })

    print("\n--- TABLA COMPARATIVA DE PREDICCIONES ---")
    print(df_resultados.to_string())

except FileNotFoundError:
    print("\nError: No se encontraron los archivos del modelo o el archivo de datos.")
except Exception as e:
    print(f"\nOcurrió un error inesperado: {e}")

Cargando el modelo local y el vectorizer...
Modelo local cargado!

Cargando el modelo pre-entrenado de Hugging Face...


Device set to use cpu


¡Modelo de Hugging Face cargado!

Realizando predicciones con ambos modelos...

--- TABLA COMPARATIVA DE PREDICCIONES ---
                                                                                                                                  Texto Original Tu Modelo Hugging Face  Confianza HF
0                                          Now I can see why Dave Winer screams about lack of Twitter API, its limitations and access throttles!  Negativo     Negativo      0.767917
1                                                                                Oooooooh... North Korea is in troubleeeee! http://bit.ly/19epAH  Positivo     Negativo      0.921869
2                                                                         @YarnThing you will not regret going to see Star Trek. It was AWESOME!  Positivo     Positivo      0.988630
3                                                                                      @XPhile1908 I have three words for you: "Safeway dot com"  Posi

### Análisis de Resultados

Al observar la tabla comparativa, se pueden extraer las siguientes conclusiones:

- **Matices y Contexto:** El modelo de Hugging Face (RoBERTa) es un Transformer, por lo que es mucho mejor para entender el contexto, el sarcasmo y los matices del lenguaje que un modelo TF-IDF. 
- **Manejo de la Neutralidad:** Este modelo solo puede clasificar como "Positivo" o "Negativo". El modelo de Hugging Face tiene una clase "Neutral", lo que le da más flexibilidad y precisión en textos que no tienen una carga sentimental clara.
- **Confianza del Modelo:** La columna `Confianza HF` indica qué tan seguro está el modelo de su predicción. En textos ambiguos, esta confianza suele ser más baja.

**Y esto muestra que estoy muy conforme con lo conseguido con mi modelo de predicción de sentimiento.**